In [21]:
import os
import kagglehub
import pandas as pd
import plotly.express as px

In [13]:
data_path = kagglehub.dataset_download("datatattle/covid-19-nlp-text-classification")

print("Path to dataset files:", data_path)


Path to dataset files: C:\Users\MotiM\.cache\kagglehub\datasets\datatattle\covid-19-nlp-text-classification\versions\1


In [25]:
train_df = pd.read_csv(os.path.join(data_path, "Corona_NLP_train.csv"), encoding="latin1")
test_df = pd.read_csv(os.path.join(data_path, "Corona_NLP_test.csv"), encoding="latin1")

In [27]:
print(train_df.shape, test_df.shape)

(41157, 6) (3798, 6)


Null count

In [26]:
train_df.isna().sum()

UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

Seems that only location has null values. So, for visualizations involving location, we'll remove them.

Number of unique classes:

In [32]:
train_df['Sentiment'].value_counts().index

Index(['Positive', 'Negative', 'Neutral', 'Extremely Positive',
       'Extremely Negative'],
      dtype='object')

In [28]:
sentiment_counts = train_df['Sentiment'].value_counts().reset_index()
sentiment_counts.columns = ['Sentiment', 'Count']
fig = px.pie(sentiment_counts, values='Count', names='Sentiment', title='Distribution of Sentiments in Training Data', hole=0.4)
fig.show()

5 classes, seems quite balanced

# Visualizations

## 1. Sentiment VS Geographical location

Clean dataframe for visualiations regarding location

In [33]:
SENTIMENT_TO_SCORE_MAPPING = {
    'Extremely Negative': -2,
    'Negative': -1,
    'Neutral': 0,
    'Positive': 1,
    'Extremely Positive': 2
    }

In [34]:
location_viz_df = train_df[~train_df["Location"].isna()]
location_viz_df["sentiment_score"] = location_viz_df["Sentiment"].map(SENTIMENT_TO_SCORE_MAPPING)

C:\Users\MotiM\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
LOCATION_IQ_TOKEN = 'pk.141e2e5711ccfbe2bea40ef7e857c721'
SAMPLE_SIZE = 300 # because of a ratelimit to the API - maximum 5K / day

In [52]:
map_viz_df = location_viz_df.sample(SAMPLE_SIZE, random_state=23)

In [53]:
import requests
import time
from tqdm import tqdm

def geocode_locationiq(location):
    url = "https://us1.locationiq.com/v1/search.php"
    params = {
        "key": LOCATION_IQ_TOKEN,
        "q": location,
        "format": "json"
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        if isinstance(data, list) and data:
            return float(data[0]['lat']), float(data[0]['lon'])
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {location}: {e}")
        return None, None

# Apply with a delay to avoid rate limits (optional, but safe)
results = []
for loc in tqdm(map_viz_df['Location']):
    lat, lon = geocode_locationiq(loc)
    results.append((lat, lon))
    time.sleep(0.5)  # Optional throttle (LocationIQ free tier = 2 req/sec)

# Split the results into new columns
map_viz_df[['latitude', 'longitude']] = pd.DataFrame(results, index=map_viz_df.index)

Error geocoding ?????: 404 Client Error: Not Found for url: https://us1.locationiq.com/v1/search.php?key=pk.141e2e5711ccfbe2bea40ef7e857c721&q=%3F%3F%3F%3F%3F&format=json


Error geocoding www.fucktrump.com: 404 Client Error: Not Found for url: https://us1.locationiq.com/v1/search.php?key=pk.141e2e5711ccfbe2bea40ef7e857c721&q=www.fucktrump.com&format=json


Error geocoding Research associate at IsAG: 429 Client Error: Too Many Requests for url: https://us1.locationiq.com/v1/search.php?key=pk.141e2e5711ccfbe2bea40ef7e857c721&q=Research+associate+at+IsAG&format=json


Error geocoding Argadahana: 404 Client Error: Not Found for url: https://us1.locationiq.com/v1/search.php?key=pk.141e2e5711ccfbe2bea40ef7e857c721&q=Argadahana&format=json


Error geocoding ??: 404 Client Error: Not Found for url: https://us1.locationiq.com/v1/search.php?key=pk.141e2e5711ccfbe2bea40ef7e857c721&q=%3F%3F&format=json


...

Error geocoding California, USA: 429 Client Error: Too Many Requests for url: https://us1.locationiq.com/v1/search.php?key=pk.141e2e5711ccfbe2bea40ef7e857c721&q=California%2C+USA&format=json


Error geocoding Probably in the kitchen: 429 Client Error: Too Many Requests for url: https://us1.locationiq.com/v1/search.php?key=pk.141e2e5711ccfbe2bea40ef7e857c721&q=Probably+in+the+kitchen&format=json


Error geocoding New Zealand (most of the time): 429 Client Error: Too Many Requests for url: https://us1.locationiq.com/v1/search.php?key=pk.141e2e5711ccfbe2bea40ef7e857c721&q=New+Zealand+%28most+of+the+time%29&format=json


Error geocoding ????????: 429 Client Error: Too Many Requests for url: https://us1.locationiq.com/v1/search.php?key=pk.141e2e5711ccfbe2bea40ef7e857c721&q=%3F%3F%3F%3F%3F%3F%3F%3F&format=json


100%|██████████| 300/300 [04:40<00:00,  1.07it/s]


In [54]:
agg_df = map_viz_df.groupby(['latitude', 'longitude'], as_index=False)['sentiment_score'].sum()
fig = px.density_mapbox(
    agg_df,
    lat="latitude",
    lon="longitude",
    z="sentiment_score",
    radius=25,
    center={"lat": 30, "lon": 0},
    zoom=1,
    mapbox_style="carto-positron"
)
fig.show()

## 2. Sentiment accross dates:
Total sentiment score per day

In [71]:
timeline_sentiment_df = train_df.copy()
timeline_sentiment_df = timeline_sentiment_df[["TweetAt", "Sentiment"]]

In [80]:
timeline_sentiment_df['TweetAt'] = pd.to_datetime(timeline_sentiment_df['TweetAt'], format='%d-%m-%Y')
heatmap_data = (
    timeline_sentiment_df.groupby([timeline_sentiment_df['TweetAt'], 'Sentiment'])
    .size()
    .reset_index(name='count')
)

# Pivot to matrix format for heatmap
heatmap_pivot = heatmap_data.pivot(index='Sentiment', columns='TweetAt', values='count').fillna(0)

heatmap_pivot = heatmap_pivot.loc[
    sorted(heatmap_pivot.index, key=lambda s: SENTIMENT_TO_SCORE_MAPPING[s])
]

# Plot heatmap
fig = px.imshow(
    heatmap_pivot,
    labels=dict(x="Date", y="Sentiment Score", color="Tweet Count"),
    x=[col.strftime('%Y-%m-%d') for col in heatmap_pivot.columns],
    y=heatmap_pivot.index,
    color_continuous_scale='YlOrRd',
    aspect='auto'
)

fig.update_layout(
    title="Sentiment Score Frequency Over Time",
    xaxis_title="Date",
    yaxis_title="Sentiment Score",
    height=400
)
fig.show()